In [ ]:
# ================================================
# Quantify Permanent and Flooded Areas
# ================================================

# --------- Install & Import Dependencies ---------
!pip install rasterio

import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import geopandas as gpd
from rasterio.features import rasterize
from google.colab import drive
import os
from scipy import ndimage

# --------- Mount Google Drive ---------
drive.mount('/content/drive')

In [ ]:
# --------- User Parameters ---------
BASE_PATH = '/content/drive/MyDrive/CONFLUENCIA/'
SHAPEFILE_PATH = BASE_PATH + 'CascoUrbanoNechi/CascoUrbanoNechi.shp'
CLASSIFIED_FOLDER = BASE_PATH + "CLASSIFIED_FILT"
OUTPUT_CSV = BASE_PATH + "filtered_flooded_area.csv"

PIXEL_AREA_M2 = 100  # 1 pixel = 100 m² (adjust if needed)

In [ ]:
# --------- Load and rasterize urban shapefile ---------
# Use first raster to get metadata
sample_raster = sorted(os.listdir(CLASSIFIED_FOLDER))[0]
with rasterio.open(CLASSIFIED_FOLDER + "/" + sample_raster) as src:
    transform = src.transform
    width, height = src.width, src.height
    raster_crs = src.crs

# Load shapefile and reproject if needed
gdf = gpd.read_file(SHAPEFILE_PATH)
if gdf.crs != raster_crs:
    gdf = gdf.to_crs(raster_crs)

# Rasterize urban mask
urban_mask = rasterize(
    [(geom, 1) for geom in gdf.geometry],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    dtype=np.uint8
)
print("✅ Urban mask rasterized.")
gdf.geometry[0]

In [ ]:
# --------- Function: get floodable zone for a given year ---------
def get_flooded_zone(year: str):
    avg_path = BASE_PATH + 'AVERAGE_CLASSIFIED_FILT/' + year + '.tif'
    with rasterio.open(avg_path) as src:
        avg = src.read(1)
        avg = avg / np.max(avg)
    permanent_water = avg > 0.75
    separated_zones = ndimage.binary_fill_holes(1 - permanent_water)
    separated_zones = 1 - ndimage.binary_fill_holes(1 - separated_zones)
    labeled, _ = ndimage.label(separated_zones)
    flood_zone = (labeled == 1) * ndimage.binary_erosion(1 - permanent_water, structure=np.ones((3, 3)))
    return flood_zone

In [ ]:
# --------- Loop: quantify for each classified raster ---------
results = []
tif_files = sorted(
    [f for f in os.listdir(CLASSIFIED_FOLDER) if f.endswith('.tif')],
    key=lambda x: (int(x.split('-')[0]), int(x.split('-')[1].split('.')[0]))
)

for tif_file in tif_files:
    year, month = tif_file.split('-')
    month = month.split('.')[0]

    with rasterio.open(CLASSIFIED_FOLDER + "/" + tif_file) as src:
        data = src.read(1)
        data = data / np.max(data)

    # Get floodable zone and mask with urban extent
    flood_zone = get_flooded_zone(year)
    urban_flood_zone = flood_zone * urban_mask

    # Extract flooded pixels within floodable zone
    flooded_in_zone = data * flood_zone

    # Filter small blobs (< 100 pixels)
    filtered_flood = np.zeros_like(flooded_in_zone)
    L = cv.connectedComponentsWithStats(flooded_in_zone.astype(np.uint8))
    stats = L[2]
    for i in range(1, L[0]):
        if stats[i][4] >= 100:
            filtered_flood[L[1] == i] = 1

    # Compute area
    flooded_area_m2 = np.sum(filtered_flood) * PIXEL_AREA_M2
    print(f"{tif_file} | Filtered Flooded Area: {flooded_area_m2:.2f} m²")

    results.append([year, month, flooded_area_m2])
    print(f"✔️ Processed: {tif_file}")